In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils.convex_sets import Singleton, Polyhedron
from utils.convex_functions import SquaredTwoNorm
from utils.graph import GraphOfConvexSets
from utils.shortest_path import ShortestPathProblem

In [ ]:
# vertices
cube = np.array([[-1, -1], [1, -1], [1, 1], [-1, 1]]) * .7
sets = [
    Singleton([0, 0]),
    Polyhedron.from_vertices(cube + np.array([2, 2])),
    Polyhedron.from_vertices(cube + np.array([4, 2])),
    Polyhedron.from_vertices(cube + np.array([2, -2])),
    Polyhedron.from_vertices(cube + np.array([4, -2])),
    Singleton([6, 0])
]
vertices = ['s', '1', '2', '3', '4', 't']

# add convex sets to the graph
G = GraphOfConvexSets()
G.add_sets(sets, vertices)
G.set_source('s')
G.set_target('t')

# edges
H = np.hstack((np.eye(2), -np.eye(2)))
l = SquaredTwoNorm(H)
edges = {
    's': ('1', '3'),
    '1': ('2',),
    '2': ('1', 't'),
    '3': ('4',),
    '4': ('3', 't')
}
for u, vs in edges.items():
    for v in vs:
        G.add_edge(u, v, l)

# Plot solution of MICP

In [ ]:
spp = ShortestPathProblem(G, relaxation=0)
sol = spp.solve()
phi = sol.primal.phi
x = sol.primal.x

print('Cost:', sol.cost)
print('\nFlows:')
for k, edge in enumerate(G.edges):
    flow = round(abs(phi[k]), 4)
    print(edge, flow)

In [ ]:
plt.figure()
G.draw_sets()
G.draw_edges()

plt.text(0, -.15, r'$X_s$', ha='center', va='top', c='k')
plt.text(2, 2.15, r'$X_1$', ha='center', va='bottom', c='k')
plt.text(4, 2.15, r'$X_2$', ha='center', va='bottom', c='k')
plt.text(2, -2.15, r'$X_3$', ha='center', va='top', c='k')
plt.text(4, -2.15, r'$X_4$', ha='center', va='top', c='k')
plt.text(6, -.15, r'$X_t$', ha='center', va='top', c='k')

kwargs = {'marker': 'o', 'markeredgecolor': 'k', 'markerfacecolor': 'w'}
G.draw_path(phi, x, color='b', linestyle='--')

# Plot solution of convex relaxation

In [ ]:
spp = ShortestPathProblem(G, relaxation=1)
sol = spp.solve()
phi = sol.primal.phi
x = sol.primal.x
y = sol.primal.y
z = sol.primal.z

print('Cost:', sol.cost)
print('\nFlows:')
for k, edge in enumerate(G.edges):
    flow = round(abs(phi[k]), 4)
    print(edge, flow)

In [ ]:
plt.figure()
G.draw_sets()

def phi(u, v):
    e = G.edges.index((u, v))
    return sol.primal.phi[e]

def y(u, v):
    e = G.edges.index((u, v))
    return sol.primal.y[e] / phi(u, v)

def z(u, v):
    e = G.edges.index((u, v))
    return sol.primal.z[e] / phi(u, v)

def plot(start, stop):
    plt.plot([start[0], stop[0]], [start[1], stop[1]], 'b--', **kwargs)
    
for e in G.edges:
    if e[0] not in [3, 4] and e[1] not in [3, 4]:
        plot(y(*e), z(*e))

plt.text(*(y('s','1') + [.2, 0]), r'$\hat y_{(s,1)}$', ha='left', va='center', c='k')
plt.text(*(z('s','1') + [0, -.1]), r'$\hat z_{(s,1)}$', ha='left', va='top', c='k')
plt.text(*(y('1','2') + [-.1, 0]), r'$\hat y_{(1,2)}$', ha='right', va='center', c='k')
plt.text(*(z('1','2') + [.1, 0]), r'$\hat z_{(1,2)}$', ha='left', va='center', c='k')
plt.text(*(y('2','1') + [.1, 0]), r'$\hat y_{(2,1)}$', ha='left', va='center', c='k')
plt.text(*(z('2','1') + [-.1, 0]), r'$\hat z_{(2,1)}$', ha='right', va='center', c='k')
plt.text(*(y('2','t') + [0, -.1]), r'$\hat y_{(2,t)}$', ha='right', va='top', c='k')
plt.text(*(z('2','t') + [-.2, 0]), r'$\hat z_{(2,t)}$', ha='right', va='center', c='k')

def center(u, v):
    return (y(u,v) + z(u,v)) / 2
plt.text(*(center('s','1') + [0, .05]), r'$0.5$', ha='right', va='bottom', c='r')
plt.text(*(center('1','2') + [0, .05]), r'$1.0$', ha='center', va='bottom', c='r')
plt.text(*(center('2','1') + [0, .05]), r'$0.5$', ha='center', va='bottom', c='r')
plt.text(*(center('2','t') + [.05, 0]), r'$0.5$', ha='left', va='bottom', c='r')